In [4]:
%pylab qt
%load_ext autoreload
%autoreload 2

[2023-12-07 14:23:13,652] {pyplot.py:414} DEBUG - Loaded backend QtAgg version 5.15.7.
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Loop through the ggems mhd files and save them to an array
import glob
import fastcat as fc
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
from mpl_interactions import ipyplot as iplt
# Zoom the ggems projections
from scipy.ndimage import zoom

ggems_scatter_files = glob.glob(
    '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/out/ggems_3e09_121kVp_*scatter.mhd'
)
ggems_primary_files = glob.glob(
    '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/out/ggems_3e09_121kVp_*[!scatter].mhd'
)

ggems_scatter_files.sort()
ggems_primary_files.sort()

ggems_primary_projections = []
ggems_scatter_projections = []

for ggems_scatter_file, ggems_primary_file in zip(ggems_scatter_files, ggems_primary_files):
    ggems_prmiary, b, c = fc.utils.read_mhd(ggems_primary_file)
    ggems_scatter, b, c = fc.utils.read_mhd(ggems_scatter_file)

    ggems_primary_projections.append(ggems_prmiary.squeeze())
    ggems_scatter_projections.append(ggems_scatter.squeeze())

# Load the pickled phantom
pkl_file = '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/ggems_1e03_121kVp.pkl'
phantom = np.load(pkl_file, allow_pickle=True)

# Convert the phantom angles to degrees
phantom_angles = phantom.angles * 180 / np.pi

# Define the denoising functions
def downsize_block(image, block_size):
    image_size = image.shape[0]
    block_size = int(block_size)
    downsize = image_size//block_size
    image = image[:downsize*block_size, :downsize*block_size]
    image = image.reshape(downsize, block_size, downsize,
                          block_size).mean(axis=(1, 3))
    return image

def denoise_projections(projections):
    projections_zoomed = []

    for projection in projections:
        projection_zoomed_downsized = downsize_block(projection, 64)
        projection_zoomed = zoom(
            projection_zoomed_downsized, 64, order=4, mode='nearest')
        # Adjust the mean of the zoomed image to match the original
        projection_zoomed = projection_zoomed * \
            np.mean(projection) / np.mean(projection_zoomed)
        projections_zoomed.append(projection_zoomed)
    
    return projections_zoomed

# zoom the scatter projections using the zoom fromfunction
ggems_scatter_denoised = denoise_projections(ggems_scatter_projections)

In [18]:
# Get the simulation parameters
simulation_parameters = phantom.simulation_parameters
detector_material = simulation_parameters['detector_material']
spectrum = simulation_parameters['spectrum']

# Load the spectrum as a fastcat spectrum object
s = fc.Spectrum()
s.load(spectrum_file=spectrum)

det = fc.Detector(s,'CsI-784-micrometer')
angles = phantom.angles

# return the projections
phantom.return_projs(det,s,angles,mgy=0, 
                    ASG=False, scat_on=False,
                    det_on=False,
                    filter_on=False,
                    return_intensity=True)# Get primary fastcat projections

[2023-12-07 15:14:11,908] {ggems_simulate.py:479} INFO -     Interp scatter 512 to 512 pixels
[2023-12-07 15:14:11,908] {ggems_simulate.py:479} INFO -     Interp scatter 512 to 512 pixels
[2023-12-07 15:14:12,090] {ggems_simulate.py:582} INFO - Running Simulations
[2023-12-07 15:14:12,091] {ggems_simulate.py:609} INFO -     Simulating 10 keV
[2023-12-07 15:14:14,906] {ggems_simulate.py:609} INFO -     Simulating 20 keV
[2023-12-07 15:14:18,133] {ggems_simulate.py:609} INFO -     Simulating 30 keV
[2023-12-07 15:14:21,341] {ggems_simulate.py:609} INFO -     Simulating 40 keV
[2023-12-07 15:14:24,690] {ggems_simulate.py:609} INFO -     Simulating 50 keV
[2023-12-07 15:14:28,197] {ggems_simulate.py:609} INFO -     Simulating 60 keV
[2023-12-07 15:14:31,817] {ggems_simulate.py:609} INFO -     Simulating 70 keV
[2023-12-07 15:14:35,469] {ggems_simulate.py:609} INFO -     Simulating 80 keV
[2023-12-07 15:14:39,021] {ggems_simulate.py:609} INFO -     Simulating 90 keV
[2023-12-07 15:14:42,576

TypeError: Phantom.return_projs() missing 3 required positional arguments: 'kernel', 'spectra', and 'angles'

In [29]:
# plt.figure()
# pli.hyperslicer(phantom.intensity, axis=0, cmap='gray')

ggems_primary_only = np.array(ggems_primary_projections) - np.array(ggems_scatter_projections)
# Set the mean and standard deviation of the ggems primary only projections to match the fastcat projections
def normalize_mean_and_std(array1,array2):
    array1 = array1 - np.mean(array1)
    array1 = array1 * np.std(array2) / np.std(array1)
    array1 = array1 + np.mean(array2)
    return array1

phantom_projections = normalize_mean_and_std(phantom.intensity, ggems_primary_only)

In [30]:
# Downsize the perc_diff using the downsize_block function
perc_diff = (np.array(ggems_scatter_denoised)-np.array(ggems_scatter_projections))/np.max(ggems_scatter_projections)*100
perc_diff_small = []
for i in range(len(perc_diff)):
    perc_diff_small.append(downsize_block(perc_diff[i], 16))

# Plot the ggems projections
fig, (axes) = plt.subplots(ncols=2, nrows=3, figsize=(5.8,10))

# Plot the denoised ggems projections
controls = pli.hyperslicer(np.array(ggems_scatter_denoised),
                           ax=axes[1][0], cmap='jet',vmax=np.percentile(np.array(ggems_scatter_denoised),99),
                           vmin=np.percentile(np.array(ggems_scatter_denoised),1))
axes[1][0].set_title('Denoised GGEMS scatter')

ggems_primary_only = np.array(ggems_primary_projections) - np.array(ggems_scatter_projections)
# Plot the ggems projections
c2 = pli.hyperslicer(np.array(ggems_primary_only), ax=axes[0][1], cmap='gray',controls=controls,
                     vmax=np.percentile(np.array(ggems_primary_only),99),
                     vmin=np.percentile(np.array(ggems_primary_only),1))
axes[0][1].set_title('GGEMS Primary')

# Plot the phantom phantom_projections
c1 = pli.hyperslicer(np.array(phantom_projections)[-1::-1], ax=axes[1][1], cmap='gray',controls=controls,
                     vmax=np.percentile(np.array(phantom_projections),99),
                     vmin=np.percentile(np.array(phantom_projections),1))
axes[1][1].set_title('Phantom Primary')

# Plot the difference between the ggems and phantom projections
perc_diff_primary = (np.array(ggems_primary_only)-np.array(phantom_projections[-1::-1]))/np.max(phantom_projections)*100

c4 = pli.hyperslicer(np.array(perc_diff_primary), ax=axes[2][1], cmap='bwr',controls=controls,
                        vmax=5,vmin=-5)
axes[2][1].set_title('Percentage Difference [%]')

# Plot the ggems projections
c2 = pli.hyperslicer(np.array(ggems_scatter_projections), ax=axes[0][0], cmap='jet',controls=controls,
                     vmax=np.percentile(np.array(ggems_scatter_projections),99),
                     vmin=np.percentile(np.array(ggems_scatter_projections),1))
axes[0][0].set_title('GGEMS Scatter')

# Plot the ggems primary projections
c3 = pli.hyperslicer(np.array(perc_diff_small),ax=axes[2][0], cmap='bwr',vmax=5,vmin=-5,controls=controls)
plt.colorbar(label='Percentage Difference [%]')
plt.tight_layout()
# Turn off the axes
for ax in axes.flatten():
    ax.axis('off')
# Define a function that given an angle and radius returns x and y coordinates
# def f(axis0,radius=220,offset=0,x0=256,y0=256,na=90):
#     angle = np.deg2rad(axis0*360/na)
#     x = radius * np.cos(angle+offset) + x0
#     y = radius * np.sin(angle+offset) + y0
#     return np.array([[x,y],[x,y],[x,y]])

# iplt.plot(f, ax=axes[4], a=(0.1,10), b=(0.1,10), parametric=True,ylim=[0,1],xlim=[1,na],play_buttons=True)
# plt.imshow(phantom.phantom[50, :, :], cmap='gray')
# iplt.scatter(f, ax=axes[3][0], controls=controls,play_buttons=True,c='r',marker='>',parametric=True,)

# Add a label in the image saying source position
# axes[3].text(50, 50, 'Source Position', color='r', fontsize=12)


[2023-12-07 15:45:37,273] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f3c20b28d90>


In [60]:
iplt.scatter?

Signature:
iplt.scatter(
    x,
    y=None,
    s=None,
    c=None,
    vmin=None,
    vmax=None,
    vmin_vmax=None,
    alpha=None,
    marker=None,
    edgecolors=None,
    facecolors=None,
    label=None,
    parametric=False,
    ax=None,
    slider_formats=None,
    xlim='stretch',
    ylim='stretch',
    force_ipywidgets=False,
    play_buttons=False,
    controls=None,
    display_controls=True,
    **kwargs,
)
Docstring:
Control a scatter plot using widgets.

Parameters
----------
x, y : function or float or array-like
    shape (n, ) for array-like. Functions must return the correct shape as well. If y is None
    then parametric must be True and the function for x must return x, y
c : array-like or list of colors or color or Callable
    Valid input to plt.scatter or a function
s : float, array-like, function, or index controls object
    valid input to plt.scatter, or a function
vmin, vmax : float, callable, shorthand for slider or indexed controls
    The vmin, vmax values

In [102]:
mean_projection = np.mean(np.array(ggems_projections), axis=0)
mean_projection_zoom = zoom(downsize_block(
    mean_projection, 64), 64, order=5, mode='nearest')

pli.heatmap_slicer(np.arange(0, mean_projection.shape[0]),
                   np.arange(0, mean_projection.shape[1]),
                   (mean_projection, mean_projection_zoom),
                   cmap='jet',
                   slices='both')

(<Figure size 1800x900 with 4 Axes>,
 array([<Axes: title={'center': 'heatmap_0'}, xlabel='X', ylabel='Y'>,
        <Axes: title={'center': 'heatmap_1'}, xlabel='X'>,
        <Axes: title={'center': 'Horizontal'}>,
        <Axes: title={'center': 'Vertical'}>], dtype=object))

In [101]:
# Do the same thing but for RectBivariateSpline
from scipy.interpolate import RectBivariateSpline
import numpy as np

ggems_projections_downsized_spline = []
for ggems_projection in ggems_projections_downsized:
    # Initialize the spline
    spline = RectBivariateSpline(np.arange(ggems_projection.shape[0]), np.arange(
        ggems_projection.shape[1]), ggems_projection, kx=4, ky=4, s=2)
    # Evaluate the spline on a finer grid
    x_fine, y_fine = np.meshgrid(np.linspace(0, ggems_projection.shape[0], ggems_projection.shape[0]*4), np.linspace(
        0, ggems_projection.shape[1], ggems_projection.shape[1]*4))
    z_fine = spline.ev(x_fine, y_fine)

    ggems_projections_downsized_spline.append(z_fine)

# Plot the ggems projections
plt.figure()
controls = pli.hyperslicer(
    np.array(ggems_projections_downsized_spline), cmap='jet')

Init signature:
RectBivariateSpline(
    x,
    y,
    z,
    bbox=[None, None, None, None],
    kx=3,
    ky=3,
    s=0,
)
Docstring:     
Bivariate spline approximation over a rectangular mesh.

Can be used for both smoothing and interpolating data.

Parameters
----------
x,y : array_like
    1-D arrays of coordinates in strictly ascending order.
    Evaluated points outside the data range will be extrapolated.
z : array_like
    2-D array of data with shape (x.size,y.size).
bbox : array_like, optional
    Sequence of length 4 specifying the boundary of the rectangular
    approximation domain, which means the start and end spline knots of
    each dimension are set by these values. By default,
    ``bbox=[min(x), max(x), min(y), max(y)]``.
kx, ky : ints, optional
    Degrees of the bivariate spline. Default is 3.
s : float, optional
    Positive smoothing factor defined for estimation condition:
    ``sum((z[i]-f(x[i], y[i]))**2, axis=0) <= s`` where f is a spline
    function. Defa

In [81]:
phantom = np.load(
    '/home/jericho/Software/fastcat/fastcat/data/phantoms/ct_scan_head_mandible_large.npy')

In [84]:
plt.figure()
contr = pli.hyperslicer(phantom)

In [85]:
# load a binary file to an array
import numpy as np

phantom = np.load()

In [86]:
plt.figure()
pli.hyperslicer(phantom.phantom)

AttributeError: 'Controls' object has no attribute 'vbox'